In [1]:
import pandas as pd

In [67]:
df=pd.read_csv('./data/local_upscaled_props_80m_cells_40_deg_grid.csv', float_precision='round_trip')

In [68]:
df.describe()

,X,Y,Z,porosity,ki,kj,kk,Is_Granitoid
count,120000.000000,120000.000000,120000.000000,120000.000000,1.200000e+05,1.200000e+05,1.200000e+05,120000.000000
mean,2000.000000,2000.000000,-340.159583,0.007056,7.012375e-17,7.101414e-17,7.697362e-17,0.744800
std,1154.474385,1154.474385,1108.276666,0.004688,7.264913e-17,7.571059e-17,7.296439e-17,0.435975
min,40.000000,40.000000,-2220.160000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,1000.000000,1000.000000,-1280.160000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
50%,2000.000000,2000.000000,-340.155000,0.008049,5.430000e-17,5.380000e-17,6.380000e-17,1.000000
75%,3000.000000,3000.000000,599.840000,0.009834,9.800000e-17,9.760000e-17,1.120000e-16,1.000000
max,3960.000000,3960.000000,1539.840000,0.028371,8.020000e-16,8.620000e-16,7.280000e-16,1.000000


Optional depending on mesh size, can do in excel for small meshes:  need to add column (1) for Id, columns 5, 6, and 7 for dX, dY, and dZ

first add the "Id"

In [69]:
df.insert(loc=0, column='Id', value=0) #need a value

now add the dX, dY, and dZ

In [70]:
mesh_size = 80

In [71]:
df.insert(loc=4, column='dX', value= mesh_size)

In [72]:
df.insert(loc=5, column='dY', value= mesh_size)

In [73]:
df.insert(loc=6, column='dZ', value= mesh_size)

now check the file

In [46]:
del df['Compressibility']

In [74]:
df.head()

,Id,X,Y,Z,dX,dY,dZ,porosity,ki,kj,kk,Is_Granitoid
0,0,40.0,3960.0,1539.84,80,80,80,0.0,0.0,0.0,0.0,0
1,0,40.0,3960.0,1459.84,80,80,80,0.0,0.0,0.0,0.0,0
2,0,40.0,3960.0,1379.84,80,80,80,0.0,0.0,0.0,0.0,0
3,0,40.0,3960.0,1299.84,80,80,80,0.0,0.0,0.0,0.0,0
4,0,40.0,3960.0,1219.84,80,80,80,0.0,0.0,0.0,0.0,0


now sort so values are in correct order for em2ex

In [75]:
a = df.sort_values(["Z","Y","X"], ascending = [True, True, True])

In [76]:
a.head()

,Id,X,Y,Z,dX,dY,dZ,porosity,ki,kj,kk,Is_Granitoid
2399,0,40.0,40.0,-2220.16,80,80,80,0.008336,5.580000e-17,7.700000e-17,9.090000e-17,1
4799,0,120.0,40.0,-2220.16,80,80,80,0.010032,1.070000e-16,1.050000e-16,9.120000e-17,1
7199,0,200.0,40.0,-2220.16,80,80,80,0.008001,4.430000e-17,6.220000e-17,6.430000e-17,1
9599,0,280.0,40.0,-2220.16,80,80,80,0.006586,1.990000e-17,3.300000e-17,3.440000e-17,1
11999,0,360.0,40.0,-2220.16,80,80,80,0.007728,6.030000e-17,4.600000e-17,8.050000e-17,1


now we have the perm and proposity all set up, but if the mesh has both granite and sediment, we need to all a column for the block number, and re-assign the perm and porosity for the sediments

this may not exist, depending on mesh size and whether it come from LF or Fracman


sediments = block 1, granite = block 2

In [77]:
a = a.rename(columns={'Is_Granitoid': 'block', 'ki': 'kii','kj':'kjj','kk':'kkk'})

In [78]:
a.block = a.block + 1

In [39]:
a = a.rename(columns={'Is_Granitoid': 'block'})

In [33]:
a.loc[a.GM == 'Granitiod', 'block'] = 2

In [34]:
a.GM.unique()

array(['Granitiod', 'Valley Fill'], dtype=object)

In [35]:
a.loc[a.GM == 'Valley Fill', 'block'] = 1

In [36]:
a.block.unique()

array([2, 1])

If needed


assign sediment properties

In [79]:
a.loc[a.block == 1, 'kii'] = 1.7e-14

In [80]:
a.loc[a.block == 1, 'kjj'] = 1.7e-14

In [81]:
a.loc[a.block == 1, 'kkk'] = 1.7e-14

In [82]:
a.loc[a.block == 1, 'porosity'] = 0.12

now can write the data to a CSV file.

In [83]:
a.to_csv("processed_local_3_40_cell.csv", encoding='utf-8', index=False)

In [13]:
a.describe().to_csv("processed_local_3_10_cell_Summary.csv")

now need to read into leapfrog and verify all the mpaaings